In [3]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

# 鍵とnonceを生成
key = get_random_bytes(32)   # AES-256用
nonce = get_random_bytes(12) # GCM推奨サイズ

# 暗号化
cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
aad = b"header-data"  # 認証のみ対象にする追加データ
cipher.update(aad)

plaintext = ("secret message"*10).encode('utf-8')
ciphertext, tag = cipher.encrypt_and_digest(plaintext)

# 暗号化結果をまとめる（Nonce + Ciphertext + Tag）
encrypted_package = nonce + ciphertext + tag
print("暗号化パッケージ:", encrypted_package)

# 復号化
nonce_dec = encrypted_package[:12]
ciphertext_dec = encrypted_package[12:-16]
tag_dec = encrypted_package[-16:]

cipher_dec = AES.new(key, AES.MODE_GCM, nonce=nonce_dec)
cipher_dec.update(aad)
decrypted = cipher_dec.decrypt_and_verify(ciphertext_dec, tag_dec)

print("復号結果:", decrypted)


暗号化パッケージ: b',6\xacR\xa3\xc9@\xca\xcb\xe3\x0e\xda\x98\xa7\xb1\x1fH\xc8\x06\xe4\xdc\xd3\xb0E\x8b\x81o\x7f\x94B$1{\xb8$\x83N\xbd\xe4\xb9\x021\x02\x1b\x9e\x97Q1\xc6x\r\x1e\xfc]P\x88\xf86\xc7\x97\xd5\xeb\xdf\xa2T\x9c\xb1\xe2\xecE\xe0\x95\xdc\x05\x05\xeb\n\xa3F\x9c#m\\\x01\x9b\xfd\x10\x02\xe5\xcb\x1a\xab\x03\xddNi\xc9vV\xee\xbaV\xf3q\x03_Q\xdf\x87!?\xc0\xf32j\xdf(\xa0\x81^X\x85hb\xc6\xaa\xd8\xf1I\xe3>\xf9\x15\xe8\xef^\xfe\xdd\x9b \xb9\xda\x86\x9b\xa9\x1ep\xcd\xebgh\xe2\x01\xc6\xcb\xf8\x1b\x08\x004\xee\x9e9=c\xe6\xbc\x0f'
復号結果: b'secret messagesecret messagesecret messagesecret messagesecret messagesecret messagesecret messagesecret messagesecret messagesecret message'


In [ ]:
import base64
encoded_str = base64.b64encode(encrypted_package).decode('utf-8')
print("Base64文字列:", encoded_str)
print(len(encoded_str))


Base64文字列: LDasUqPJQMrL4w7amKexH0jIBuTc07BFi4Fvf5RCJDF7uCSDTr3kuQIxAhuel1ExxngNHvxdUIj4NseX1evfolScseLsReCV3AUF6wqjRpwjbVwBm/0QAuXLGqsD3U5pyXZW7rpW83EDX1HfhyE/wPMyat8ooIFeWIVoYsaq2PFJ4z75FejvXv7dmyC52oabqR5wzetnaOIBxsv4GwgANO6eOT1j5rwP


NameError: name 'encode' is not defined